### 멜론 실시간 100곡 크롤링 하기

In [1]:
# 실시간 url 체크 및 header 파악
url = 'https://www.melon.com/chart/index.htm'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}

In [2]:
# server로 요청
response = requests.get(url, headers = header)
#html 파일이므로, beautifulsoup을 이용하여 파싱
dom = BeautifulSoup(response.content,'html.parser')
# 1위 ~ 50위와 51위 ~ 100위의 id가 다르므로, 다른 변수에 담음
titles1 = dom.select('#lst50 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > [title]')
titles2 = dom.select('#lst100 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > [title]')
artists1 = dom.select('#lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > .checkEllipsis')
artists2 = dom.select('#lst100 > td:nth-child(6) > div > div > div.ellipsis.rank02 > .checkEllipsis')
imgs1 = dom.select('#lst50 > td:nth-child(4) > div > a > img')
imgs2 = dom.select('#lst100 > td:nth-child(4) > div > a > img')

In [3]:
# 원하는 값을 제대로 가져오는지 확인
title1 = titles1[0].text
title2 = titles2[0].text
artist1 = artists1[0].text
artist2 = artists2[0].text
song_id1 = titles1[0].get('href').split(',')[-1].split(')')[0]
song_id2 = titles2[0].get('href').split(',')[-1].split(')')[0]
album_art_url1 = imgs1[0].get('src')
album_art_url2 = imgs2[0].get('src')

In [4]:
# 100개 크롤링하여 데이터프레임으로 변경

top100 = []

for i in range(0,50):
    top100.append({
    'title' : titles1[i].text,
    'artist' : artists1[i].text,
    'song_id' : titles1[i].get('href').split(',')[-1].split(')')[0],
    'album_art_url' : imgs1[i].get('src'),
    })
    

for i in range(0,50):
    top100.append({
    'title' : titles2[i].text,
    'artist' : artists2[i].text,
    'song_id' : titles2[i].get('href').split(',')[-1].split(')')[0],
    'album_art_url' : imgs2[i].get('src')
    })
top100 = pd.DataFrame(top100)
top100

,title,artist,song_id,album_art_url
0,ON,방탄소년단,32399830,https://cdnimg.melon.co.kr/cm2/album/images/10...
1,아무노래,지코 (ZICO),32313543,https://cdnimg.melon.co.kr/cm2/album/images/10...
2,마음을 드려요,아이유,32378104,https://cdnimg.melon.co.kr/cm2/album/images/10...
3,00:00 (Zero O’Clock),방탄소년단,32399832,https://cdnimg.melon.co.kr/cm2/album/images/10...
4,Black Swan,방탄소년단,32323969,https://cdnimg.melon.co.kr/cm2/album/images/10...
...,...,...,...,...
95,사계 (Four Seasons),태연 (TAEYEON),31688205,https://cdnimg.melon.co.kr/cm/album/images/102...
96,잘 살고 있다,V.O.S,32225055,https://cdnimg.melon.co.kr/cm2/album/images/10...
97,워커홀릭,볼빨간사춘기,32038651,https://cdnimg.melon.co.kr/cm2/album/images/10...
98,Know Me Too Well,"New Hope Club, Danna Paola",32077651,https://cdnimg.melon.co.kr/cm2/album/images/10...


In [5]:
# 이미지 저장
import re
import string

In [6]:
# 이미지 url 확인
re.sub('resize/120/quality/80/optimize','',top100['album_art_url'][0])

'https://cdnimg.melon.co.kr/cm2/album/images/103/77/346/10377346_20200221153622_500.jpg/melon/'

In [7]:
# 이미지 저장될 폴더 생성
!mkdir melontop100

하위 디렉터리 또는 파일 melontop100이(가) 이미 있습니다.


In [8]:
# download 함수 생성
def download(title, litk):
    response = requests.get(link, stream=True)
    path = f'./melontop100/{title}'

    with open(path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [9]:
# 반복문으로 이미지 크롤링
for idx, row in top100.iterrows():
    link = re.sub('resize/120/quality/80/optimize','',row['album_art_url'])
    title = row['title'] + '.png'
    title = title.replace(':','').replace('/','').replace('"','')
    print(title, link)
    download(title, link)
print('완료')

ON.png https://cdnimg.melon.co.kr/cm2/album/images/103/77/346/10377346_20200221153622_500.jpg/melon/
아무노래.png https://cdnimg.melon.co.kr/cm2/album/images/103/75/118/10375118_20200113150502_500.jpg/melon/
마음을 드려요.png https://cdnimg.melon.co.kr/cm2/album/images/103/88/744/10388744_20200214175740_500.jpg/melon/
0000 (Zero O’Clock).png https://cdnimg.melon.co.kr/cm2/album/images/103/77/346/10377346_20200221153622_500.jpg/melon/
Black Swan.png https://cdnimg.melon.co.kr/cm2/album/images/103/77/346/10377346_20200221153622_500.jpg/melon/
친구.png https://cdnimg.melon.co.kr/cm2/album/images/103/77/346/10377346_20200221153622_500.jpg/melon/
METEOR.png https://cdnimg.melon.co.kr/cm2/album/images/103/59/162/10359162_500.jpg/melon/
FIESTA.png https://cdnimg.melon.co.kr/cm2/album/images/103/89/281/10389281_20200217162236_500.jpg/melon/
작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey).png https://cdnimg.melon.co.kr/cm/album/images/102/73/641/10273641_500.jpg/melon/
Filter.png https://cdnimg.melon.co.kr/cm2/a